In [1]:
# Language Delay Proxy Detection (No Audio)
import cv2
import mediapipe as mp
import time

# Initialize Pose Estimator
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# State Tracking
gesture_detected = False
gesture_response_time = None
start_time = None
response_timeout = 4  # seconds

cap = cv2.VideoCapture(0)
print("Wave your hand in front of the camera to simulate gesture trigger.")
print("Press 'q' to quit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    frame_height, frame_width = frame.shape[:2]

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Get wrist and shoulder positions
        left_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]

        # If wrist is clearly above shoulder = waving gesture
        if left_wrist.y < left_shoulder.y:
            if not gesture_detected:
                gesture_detected = True
                start_time = time.time()
                print("Gesture Detected – Waiting for response...")

        # Simulate toddler turning head (using nose X-coordinate)
        if gesture_detected:
            nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
            nose_x = nose.x

            if nose_x < 0.4 or nose_x > 0.6:  # assume child turns head
                gesture_response_time = time.time() - start_time
                print(f"Response time: {gesture_response_time:.2f} sec")
                if gesture_response_time > response_timeout:
                    cv2.putText(frame, "⚠️ Possible Language Delay", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                else:
                    cv2.putText(frame, "✅ Normal Response", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                gesture_detected = False

    # Display
    cv2.imshow("Language Delay Proxy Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'mediapipe'